In [2]:
import numpy as np
from numba import jit
from scipy.stats.mstats import mquantiles 

# Optimizing Numpy Performance - Example VaR Calculation

## VaR

In [3]:
pl = np.random.rand(5000, 150)

In [4]:
cl = 1 - 0.95

In [5]:
def quan(pl):
    return mquantiles(pl, cl, alphap=0, betap=0, axis=1)

In [6]:
%time var = quan(pl)

CPU times: user 1.74 s, sys: 3 ms, total: 1.74 s
Wall time: 1.74 s


In [7]:
var.shape

(5000, 1)

In [25]:
%%time
pl_gen = (np.random.rand(5000, 150) for i in range(10))
res = [quan(pl) for pl in pl_gen]

CPU times: user 17.5 s, sys: 9 ms, total: 17.5 s
Wall time: 17.5 s


In [8]:
import concurrent.futures as fut

In [115]:
%%time
pl_gen = (np.random.rand(5000, 150) for i in range(10))
with fut.ProcessPoolExecutor() as ex:
    futures = [ex.submit(quan, pl) for pl in pl_gen]
    res = [future.result() for future in futures]

CPU times: user 174 ms, sys: 180 ms, total: 354 ms
Wall time: 5.8 s


## Subwindow VaR

### Sinlge Trade

In [9]:
pl1 = pl[0]

In [10]:
window = 60

In [112]:
%%timeit
cba_var1 = []
for i in range(len(pl1) - window + 1):
    sw = pl1[i:i+window]
    cba_var1.append(mquantiles(sw, cl, 0, 0)[0])

25.6 ms ± 74.5 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Pure Python approach for creating subwindows

In [32]:
def rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

In [113]:
%timeit cba_var2 = quan(rolling_window(pl1, window))

31.4 ms ± 40.7 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Vectorized subwindow creation is actually slower, not faster here.

In [94]:
@jit(nopython=True)
def calc_cba(pl, window):
    nr_subwindows = len(pl) - window + 1
    cba_pls = np.zeros((nr_subwindows, window))
    for i in range(nr_subwindows):
        cba_pls[i, :] = pl[i:i+window]
    return cba_pls

In [114]:
%%timeit
cba_pls = calc_cba(pl1, window)
cba_var3 = quan(cba_pls)

31.5 ms ± 88.7 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [117]:
cba_pls.shape

(91, 60)

Subwindow creation with Numba is also not faster.

### All Trades

In [12]:
@jit(nopython=True)
def calc_cba2(pls, window):
    nr_pl = pls.shape[0]
    nr_subwindows = len(pls[0, :]) - window + 1
    cba_pls = np.zeros((nr_pl * nr_subwindows, window))
    for j in range(nr_pl):
        pl = pls[j, :]
        for i in range(nr_subwindows):
            cba_pls[i + j*nr_subwindows, :] = pl[i:i+window]
    return cba_pls

In [21]:
%time cba_pls2 = calc_cba2(pl, window)

CPU times: user 72 ms, sys: 112 ms, total: 184 ms
Wall time: 184 ms


Creating subwindows is actually not the bottleneck, even pure Python. Using Numba gives a speedup by a factor of 10.

In [123]:
%%time
cba_pls2 = calc_cba2(pl, window)
cba_var4 = quan(cba_pls2)

CPU times: user 2min 34s, sys: 199 ms, total: 2min 34s
Wall time: 2min 34s


Quantile calculation using Scipy mquantiles takes a very long time for a large number of PL vectors.

In [127]:
def quant_par(pl, chunksize=1000):
    with fut.ProcessPoolExecutor() as ex:
        futures = []
        for i in range(pl.shape[0] // chunksize):
            chunk = pl[i*chunksize:(i+1)*chunksize, :]
            futures.append(ex.submit(quan, chunk))
        res = np.concatenate([future.result() for future in futures])        

In [133]:
%%time 
cba_pls2 = calc_cba2(pl, window)
cba_va5 = quant_par(cba_pls2)

CPU times: user 1.29 s, sys: 413 ms, total: 1.7 s
Wall time: 45.1 s


In [17]:
cba_pls2.shape

(455000, 60)

Parallelization gives the expected speedup.

But could we get faster?

In [22]:
%time cba_pls2.sort()

CPU times: user 971 ms, sys: 5 ms, total: 976 ms
Wall time: 973 ms


Sorting the array is actually quite fast.

ToDo: reimplement mquantiles function in a faster way (vectorized and/or numba).